## Install Lib API

In [1]:
! pip install https://dnaink.jfrog.io/artifactory/dna-ink-pypi/model-fkeywords/0.1.0/model_fkeywords-0.1.0-py3-none-any.whl

     |████████████████████████████████| 62 kB 645 kB/s 
     |████████████████████████████████| 274 kB 8.9 MB/s 
     |████████████████████████████████| 12.7 MB 27.9 MB/s 
     |████████████████████████████████| 1.1 MB 63.1 MB/s 
     |████████████████████████████████| 212.3 MB 14 kB/s 
     |████████████████████████████████| 285 kB 49.9 MB/s 
     |████████████████████████████████| 1.5 MB 55.8 MB/s 
     |████████████████████████████████| 141 kB 54.4 MB/s 
     |████████████████████████████████| 366 kB 66.2 MB/s 
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 7.9 MB 29.2 MB/s 
     |████████████████████████████████| 2.1 MB 39.2 MB/s 
     |████████████████████████████████| 748 kB 61.8 MB/s 
     |████████████████████████████████| 489 kB 75.1 MB/s 
     |████████████████████████████████| 198 kB 75.0 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |██████████████

In [2]:
! python -m spacy download pt_core_news_sm

     |████████████████████████████████| 22.1 MB 311 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


## Import libs

In [1]:
import pandas as pd
import spacy
import nltk
nltk.download('stopwords')
from api_model import nlextract

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [2]:
extractor = nlextract.NLExtractor()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read Excel file

In [3]:
df = pd.read_excel("/content/Conversas Sodexo.xlsx", engine='openpyxl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179685 entries, 0 to 179684
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Codigo                 179685 non-null  int64         
 1   CHAT_Protocolo         179685 non-null  int64         
 2   TEM                    179685 non-null  int64         
 3   ChatID                 179685 non-null  int64         
 4   ExpertID               179685 non-null  int64         
 5   ExpertFullName         179685 non-null  object        
 6   CategoryName           179685 non-null  object        
 7   ClosedBy               179614 non-null  object        
 8   DateAnswered           179685 non-null  float64       
 9   DateFinished           179541 non-null  float64       
 10  CustomerID             179685 non-null  int64         
 11  CustomerName           179644 non-null  object        
 12  CustomerEmail          179685 non-null  obje

#### See the first line of dataset

In [4]:
df.head(1)

,Codigo,CHAT_Protocolo,TEM,ChatID,ExpertID,ExpertFullName,CategoryName,ClosedBy,DateAnswered,DateFinished,CustomerID,CustomerName,CustomerEmail,DataInsercao,Status,Resolution,CustomerFieldA,CustomerFieldC,CustomerFieldD,CustomerFieldE,CustomerFieldG,CustomerFieldH,CustomerFieldU,Link,UniqueID,Type,MsgTipo,ClassificacaoTags,ClassificacaoTreeName,CustomerName_1,ExpertFullName_2,Date,MÊS,Message,Column1
0,7,350229426,350229426,31646020,202084,DAINA ANDREZA DE SOUZA,Chat - Atendimento Robô,Encerrado pelo consumidor,44470.349259,44470.354734,80596540,AMANDA,administrativo@grupoapec.com.br,44559.410706,2,2.0,7075888000170,NaN,NaN,NaN,NaN,NaN,NaN,sodexoa,3.164602e+16,1201,1201,NaN,NaN,AMANDA,NaN,2021-10-01 08:22:56,OUTUBRO,Bloqueio de Cartão,NaN


### Limpeza de acentuações

#### Atribuir o nome original da coluna de texto para uma variavel (pode se usar qualquer coluna que tenha texto)

In [5]:
coluna_texto = 'Message'

In [7]:
df[f'{coluna_texto}_clean'] = df[coluna_texto].apply(extractor.udf_clean_text) ## limpa toda a acentuação, caracteres especiais,
                                                                                  # pontuações do texto.

In [8]:
df[[f'{coluna_texto}_clean', coluna_texto]].head(10)

,Message_clean,Message
0,bloqueio de cartao,Bloqueio de Cartão
1,ola seja bemvindo a ao chat da sodexo por favor anote o numero do seu protocolo para iniciarmos ah e nao esqueca de avaliar ao final do atendimento wink,"Olá, seja bem-vindo (a) ao Chat da Sodexo. Por favor anote o número do seu protocolo 350229426, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:"
2,bom dia,Bom dia
3,gostaria de solicitar o desbloqueio de dois cartoes,gostaria de solicitar o desbloqueio de dois cartões
4,confirmacao de dados para pedido,Confirmação de dados para pedido
5,ola seja bemvindo a ao chat da sodexo por favor anote o numero do seu protocolo para iniciarmos ah e nao esqueca de avaliar ao final do atendimento wink,"Olá, seja bem-vindo (a) ao Chat da Sodexo. Por favor anote o número do seu protocolo 350229610, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:"
6,bom dia,bom dia
7,preciso saber sobre um pedido que foi feito a mais de uma semana,preciso saber sobre um pedido que foi feito á mais de uma semana
8,nº,nº - 26281401/21
9,bom dia amanda tudo bem,"Bom dia, Amanda tudo bem?"


### Call the functions inside of wheels file

#### Lista de Palavras Procuradas

#### Formato Exemplo:
  variavel = {coluna_nome : [lista de palavras procuradas],
              coluna_nome : [lista de palavras procuradas]}

In [9]:
mydict = {
    'alto_atrito': ['ainda não', 'não chegou', 'até agora', 'até o momento', 'atraso', 'reclamação', 'pelo amor de Deus', 'procon', \
                   'péssimo', 'cansado', 'incompetente', 'ouvidoria', 'frustração', 'absurdo', 'porra', 'PQP', 'poxa vida', 'horrível',\
                   'ridículo', 'decepção', 'humilhação', 'FDP', 'merda', 'triste', 'bosta', 'protocolo','so um miniuto'],
    'cancelamento': ['cancelar minha viagem', 'quero cancelar', 'cancelamento', 'desejo cancelar', 'quero desativar', 'verificado'],
    'callback': ['me liga de novo', 'está me ligado']
          }

#### Colocar coluna original para lowercase (minusculo)

In [10]:
df[coluna_texto] = df[coluna_texto].str.lower()

#### Aplicar função de Caça Palavras no Texto

In [11]:
coluna_texto = f'{coluna_texto}_clean'

In [12]:
try:
  for key in mydict:
    df[key] =  df[coluna_texto].apply(lambda x: extractor.pattern_matcher(x,mydict[key],mode="dictionary", anonymizer=False,custom_anonymizer=f'<{key}>'))
except IOError as e:
  print(f'não tem mais listas para rodar dados {e}')
  pass

In [13]:
df[['Message','Message_clean','alto_atrito','cancelamento','callback']].head(20)

,Message,Message_clean,alto_atrito,cancelamento,callback
0,bloqueio de cartão,bloqueio de cartao,[],[],[]
1,"olá, seja bem-vindo (a) ao chat da sodexo. por favor anote o número do seu protocolo 350229426, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:",ola seja bemvindo a ao chat da sodexo por favor anote o numero do seu protocolo para iniciarmos ah e nao esqueca de avaliar ao final do atendimento wink,"[{'entity': 'protocolo', 'value': 'protocolo', 'type': 'protocolo', 'startIndex': 69, 'endIndex': 79}]",[],[]
2,bom dia,bom dia,[],[],[]
3,gostaria de solicitar o desbloqueio de dois cartões,gostaria de solicitar o desbloqueio de dois cartoes,[],[],[]
4,confirmação de dados para pedido,confirmacao de dados para pedido,[],[],[]
5,"olá, seja bem-vindo (a) ao chat da sodexo. por favor anote o número do seu protocolo 350229610, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:",ola seja bemvindo a ao chat da sodexo por favor anote o numero do seu protocolo para iniciarmos ah e nao esqueca de avaliar ao final do atendimento wink,"[{'entity': 'protocolo', 'value': 'protocolo', 'type': 'protocolo', 'startIndex': 69, 'endIndex': 79}]",[],[]
6,bom dia,bom dia,[],[],[]
7,preciso saber sobre um pedido que foi feito á mais de uma semana,preciso saber sobre um pedido que foi feito a mais de uma semana,[],[],[]
8,nº - 26281401/21,nº,[],[],[]
9,"bom dia, amanda tudo bem?",bom dia amanda tudo bem,[],[],[]


## Caça Palavras com palavras exatas do texto

In [14]:
mydict = {
    'alto_atrito_2': ['ainda não', 'não chegou', 'até agora', 'até o momento', 'atraso', 'reclamação', 'pelo amor de Deus', 'procon', \
                   'péssimo', 'cansado', 'incompetente', 'ouvidoria', 'frustração', 'absurdo', 'porra', 'PQP', 'poxa vida', 'horrível',\
                   'ridículo', 'decepção', 'humilhação', 'FDP', 'merda', 'triste', 'bosta', 'protocolo'],
    'cancelamento_2': ['cancelar minha viagem', 'quero cancelar', 'cancelamento', 'desejo cancelar', 'quero desativar', 'verificado']
          }

In [15]:
try:
  for key in mydict:
    df[key] =  df[coluna_texto].apply(lambda x: extractor.udf_type_keywords(x,mydict[key],mode="dictionary"))
except IOError as e:
  print(f'não tem mais listas para rodar dados {e}')
  pass

#### Avaliação das colunas do Caça Palavras geradas

In [16]:
df[['Message','Message_clean','alto_atrito','cancelamento','alto_atrito_2', 'cancelamento_2']].head(50)

,Message,Message_clean,alto_atrito,cancelamento,alto_atrito_2,cancelamento_2
0,bloqueio de cartão,bloqueio de cartao,[],[],,
1,"olá, seja bem-vindo (a) ao chat da sodexo. por favor anote o número do seu protocolo 350229426, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:",ola seja bemvindo a ao chat da sodexo por favor anote o numero do seu protocolo para iniciarmos ah e nao esqueca de avaliar ao final do atendimento wink,"[{'entity': 'protocolo', 'value': 'protocolo', 'type': 'protocolo', 'startIndex': 69, 'endIndex': 79}]",[],protocolo,
2,bom dia,bom dia,[],[],,
3,gostaria de solicitar o desbloqueio de dois cartões,gostaria de solicitar o desbloqueio de dois cartoes,[],[],,
4,confirmação de dados para pedido,confirmacao de dados para pedido,[],[],,
5,"olá, seja bem-vindo (a) ao chat da sodexo. por favor anote o número do seu protocolo 350229610, para iniciarmos... ah, e não esqueça de avaliar ao final do atendimento! :wink:",ola seja bemvindo a ao chat da sodexo por favor anote o numero do seu protocolo para iniciarmos ah e nao esqueca de avaliar ao final do atendimento wink,"[{'entity': 'protocolo', 'value': 'protocolo', 'type': 'protocolo', 'startIndex': 69, 'endIndex': 79}]",[],protocolo,
6,bom dia,bom dia,[],[],,
7,preciso saber sobre um pedido que foi feito á mais de uma semana,preciso saber sobre um pedido que foi feito a mais de uma semana,[],[],,
8,nº - 26281401/21,nº,[],[],,
9,"bom dia, amanda tudo bem?",bom dia amanda tudo bem,[],[],,


## Salvar o Resultado em um CSV

In [17]:
filename = 'sodexo_tratado'
file_save = f'{filename}.csv'
df.to_csv(file_save, sep=';',encoding='utf-8',index=False)